## What is Tensorflow?
[Tensorflow](https://www.tensorflow.org/) (TF) is a software library created by Google that you can use to define complex mathematical functions and automatically compute their derivatives. Somewhat obvious by its name, one of its strengths is that it can works well with tensors (similar to how MatLab is short for Matrix Laboratory). 

Defining functions in TF (and some other machine learning libraries such as [Theano](http://deeplearning.net/software/theano/)) is somewhat different from how programmers conventionally define functions. In a traditional program, you define functions and then use them on your data. E.g. $$ c=a*b $$

### Using Strictly Python

In [1]:
def multiply(a, b): return a*b
x = 4
y = 5
print (multiply(x,y))

20


However, in TF, you define a computational graph which you later evaluate using your data. E.g.

### Using Tensorflow

In [2]:
import tensorflow as tf
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
mulitply = a * b
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run([mulitply], {a:4.0, b:5.0}))
sess.close()

[20.0]


This seems far more complicated, doesn't it? There are a few things to notice. 

1. Variables of type `tf.placeholder()` with corresponding datatypes `tf.float32` had to be defined for every input to our function (or operation).
2. Something called a `tf.Session()` had to be defined and was used for all subsequent computations. Afterwards it was closed.
3. Variables had to be initialized.
4. The operation `mulitply` had to be fed in with corresponding inputs to `sess.run()`.
5. The session returned our answer in an array of length 1.

So what are the benefits? Well, TF has a large library of available mathematical functions to use and it can automatically compute the derivates. As functions get more complicated, this becomes a godsend.

## Let's explore some of these components in a little more detail

### Inputs and Variables
In TF, you aren't defining functions as your normally would. Instead, you are defining a computational graph which stores information on variables and the operations they will go through. The two main types of variables you want to keep track of are
1. **`tf.placeholder()`**
    1. These are used to define inputs fors your computational graph which you can use to supply your own data. TF is similar to languages like C++ in that (at least input) variable's data types must be defined when they declared (e.g. `tf.float32`). [Here is a list of TF data types](https://www.tensorflow.org/resources/dims_types).
2. **`tf.Variable()`**
    1. These are in-memory bffers that contain tensors which hold and maintain parameters for your model. They must be explicitly initialized and can be saved to disk during or after training. For example, if you're training a neural network, a `tf.Variable()` will likely encapsulate the matrices which define the forward progogation across a layer of your network.

There are a few useful things to knows that I will go over below.

#### Variables have to be initialized

In [3]:
m_p = tf.Variable(tf.ones((2,2)), name = "variable_matrix") # note that variables are defined similarly to numpy
m = tf.zeros((2,2))
sess = tf.Session()
print (sess.run(m))
sess.run(tf.global_variables_initializer()) # if you comment this out, this will not run with 
                                            # FailedPreconditionError: Attempting to use uninitialized value
print (sess.run(m_p))
sess.close()

[[ 0.  0.]
 [ 0.  0.]]
[[ 1.  1.]
 [ 1.  1.]]


#### TF provides global access to variables and facilitates the process with variable scopes
Your model may end up having many variables that it depends on. For a neural network, you could for example have a matrix and a bias vector for each layer. If you have 7 layers, that's 14 variables. TF makes it easy to easily access these variables with a combination of variable scopes and global access via `tf.variable_scope()` and `tf.get_variable()`, respectively. Below I will define a simple 3 layer neural network

In [4]:
def neural_network(X):
    layer1 = tf.Variable(tf.random_normal([3,3], stddev=0.1), name="matrix1")
    bias1 = tf.Variable(tf.zeros([1]), name="bias1")
    output1 = tf.matmul(X, matrix1) + bias1

    layer2 = tf.Variable(tf.random_normal([3,3], stddev=0.1), name="matrix1")
    bias2 = tf.Variable(tf.zeros([1]), name="bias2")

    layer3 = tf.Variable(tf.random_normal([3,3], stddev=0.1), name="matrix1")
    bias3 = tf.Variable(tf.zeros([1]), name="bias2")

    return tf.matmul(m1, matrix2)

## new
def forward_pass(input, matrix_shape, bias_shape):
    matrix = tf.get_variable(tf.random_normal(matrix_shape, stddev=0.1), name="matrix")
    bias = tf.get_variable(tf.zeros(bias_shape), name="bias")
    return tf.matmul(input, matrix1) + bias1

def function(X):
    with tf.variable_scope("matrix1"):
        output = matrix_multiplication(X, [3, 3], [3])
    with tf.variable_scope("matrix2"):
        return matrix_multiplication(output, [3, 3], [3])


#### Now your model may have many models to track. To make things easier, TF provides `tf.variable_scope()` to make this easier.

### Useful Links

1. [Stanford CS224 Introduction to Tensorflow](https://cs224d.stanford.edu/lectures/CS224d-Lecture7.pdf)
1. [Tensorflow Sessions](https://www.tensorflow.org/api_docs/python/client/session_management#Session)
1. [Tensorflow Datatypes](https://www.tensorflow.org/resources/dims_types)
1. [Tensorflow Variables](https://www.tensorflow.org/api_docs/python/state_ops/variables#Variable)
1. [Tensorflow Variable Scopes](https://www.tensorflow.org/versions/master/how_tos/variable_scope/)

# Deleted
1. **Variables of type (`tf.placeholder()`) with corresponding datatypes (`tf.float32`) had to be defined for every input to our function (or operation). **
  1. In TF, you are defining a graph which contains all the operations that will be performed on data and their order. `tf.placeholder()` is used to define inputs into the computational graph which you will later use as entry points for your data. TF is similar to languages like C++ in that a variable's data type must be defined when it is declated (e.g. `tf.float32`). [Here is a list of TF data types](https://www.tensorflow.org/resources/dims_types).

2. ** Something called a `tf.Session()` had to be defined and was used for all subsequent computations. Afterwards it was closed.**
  1. TF uses sessions to encapsulate the environment in which operations are executed and variable values are stored. By default, you only have one session running. However, you could run multiple sessions. Possibly utility? If you have a graph which defines operations by a neural network, you can simulate multiple learning instances of this neural network in different threads with their own corresponding sessions.

3. **Variables had to be initialized.**

4. **The operation (`c`) had to be fed in with corresponding data to `sess.run()`.**
  1. TF will use `sess.run()` to run the defined computational graph to the point defined by the given operation. A dictionary variables and their values must be fed into `sess.run()`. The keys may either be the TF variables or their names. I will demonstrate this later.

5. **The session returned our answer in an array of length 1.**
  1. TF works with Tensors. Arrays are 1D tensors. Matrices are 2D tensors. Everything you get will be in the form of a tensor. As you start building graphs, tensor shapes will become very important. Be careful as TF tends to be finnicky.

So what are the benefits? I see two primary ones. TF has a large library of available mathematical functions to use and it can automatically compute the derivates. As functions get more complicated, this becomes more of a godsend.